In [ ]:
import numpy as np
import robustsp as rsp

def m_param_est(receive, C, Theta, parameter):
    Pseudoinv = np.linalg.pinv(C)
    residuals = receive - C@Theta
    Theta = [*np.zeros(len(Theta[:,0])), *Theta , *np.zeros(len(Theta[:,0]))]
    kk = 0
    mad = lambda x: np.mean(np.abs(x-np.mean(x)))
    noisescale = 1.483 *mad(residuals)
    
    thresh = np.sum(np.abs((Theta[:,kk+1]-Theta[:,kk])/Theta[:,kk+1]))
    while thresh>parameter['break']:
        
        z, Phi_point = rsp.asymmetric_tanh(residuals/noisescale,parameter['c1'],parameter['c2'],parameter['x1'])
        kk += 1
        muu = 1.25* np.max(np.abs(Phi_point))
        
        # update parameter estimate
        Theta[:,kk+1] = Theta[:,kk] + 1/muu * Pseudoinv * z * noisescale
        
        residuals = receive - C @ Theta[:,kk+1]
        noisescale = 1.483*mad(residuals)
        
        if kk >parameter['max_iters']:
            break
        
    th2 = Theta[:,kk+1]
    
    return th2, Theta, kk, residuals